In [49]:
import nltk
from nltk import FreqDist
nltk.download('stopwords') 

import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import re
import spacy

import gensim
from gensim import corpora,models

from itertools import chain



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sphulse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
from itertools import chain

In [51]:
nlp = spacy.load(r'C:\Users\sphulse\AppData\Local\Continuum\anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.0.0', disable=['parser', 'ner'])

def lemmatization(texts, tags=['NOUN', 'ADJ']): # filter noun and adjective
       output = []
       for sent in texts:
             doc = nlp(" ".join(sent)) 
             output.append([token.lemma_ for token in doc if token.pos_ in tags])
       return output

In [52]:
df = pd.read_csv('GTD_Final_Reviews.csv',index_col=None)

book_indicators=['book','gtd','productivity','audible','david','allen','tasks','read',
                 'write','author','boring','information','system','plan','focus',
                'schedule','practice','busy']

#~(df.Comment.apply(lambda review: any(word in review for word in book_indicators)))
df = df[(~df.Comment.isnull())]
df = df[(~(df.Comment.apply(lambda review: any(word in review.lower() for word in book_indicators))))]
df.shape



(1115, 6)

In [53]:
df.head()

,Comment,Comment_Title,Date,User Rating,User_Account_Link,Username
0,"We love this product, easy to use, very fast expand and seems good quality.",Ease of use.,"July 23, 2019",5.0,/gp/profile/amzn1.account.AETH36IYWODOTVA42WDH4AFSGWKA,Amazon Customer
1,My 14yr. Old son loves this flag. It is the perfect size for above his bed. Also is great of great quality and seems to be durable.,Well worth the price.,"July 21, 2019",5.0,/gp/profile/amzn1.account.AFL2SCPMVJ4E522GFQ2TTBXOFURQ,patti walsh
3,Only time will tell but this is the fifth pocket hose I have purchased and this one seems sturdier than the others.,Seems like a sturdy expandable hose,"July 19, 2019",5.0,/gp/profile/amzn1.account.AGBDIUPNRKS5DTRPFO5O5BJLHAUA,Kathy Kostka Constantine
4,Gaarden,Great Hose,"July 18, 2019",4.0,/gp/profile/amzn1.account.AFQYX7YCA2AGG4DKW3WL3UKVINZQ,Amazon Customer
6,"I purchased the 25', the 100' and now the 75'. They are all wonderful, but my favorite one is the 75', as it allows me towash the car and do everything I need it to do without being too short or l...",Best Expandable Garden Hose Ever,"July 15, 2019",5.0,/gp/profile/amzn1.account.AH76W5YGQ63C33NMCRSKYYI4JB6A,Khorto


In [54]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

documents=df['Comment'].copy()

# remove unwanted characters, numbers and symbols
df['Comment'] = df['Comment'].str.replace("[^a-zA-Z#]", " ")

# function to remove stopwords
def remove_stopwords(rev):
    rev_new = " ".join([i for i in rev if i not in stop_words])
    return rev_new

# remove short words (length < 3)
df['Comment'] = df['Comment'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# remove stopwords from the text
reviews = [remove_stopwords(r.split()) for r in df['Comment']]

# make entire text lowercase
reviews = [r.lower() for r in reviews]


tokenized_reviews = pd.Series(reviews).apply(lambda x: x.split())
print(tokenized_reviews[1])

reviews_2 = lemmatization(tokenized_reviews)
print(reviews_2[1]) # print lemmatized review




['old', 'son', 'loves', 'flag', 'perfect', 'size', 'bed', 'also', 'great', 'great', 'quality', 'seems', 'durable']
['old', 'son', 'flag', 'perfect', 'size', 'bed', 'great', 'great', 'quality', 'durable']


In [55]:
# remove words that appear only once
all_tokens = sum(reviews_2, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
texts = [[word for word in text if word not in tokens_once] for text in reviews_2]

In [56]:
# Create Dictionary.
id2word = corpora.Dictionary(texts)
# Creates the Bag of Word corpus.
mm = [id2word.doc2bow(text) for text in texts]

# Trains the LDA models.
lda = models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=12, \
                               update_every=1, chunksize=10000, passes=1,minimum_probability=0)

# Prints the topics.
for top in lda.print_topics():
  print(top)
print

# Assigns the topics to the documents in corpus
lda_corpus = lda[mm]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)
print(threshold)


(0, '0.060*"product" + 0.034*"good" + 0.030*"use" + 0.028*"hose" + 0.026*"quality" + 0.019*"love" + 0.019*"happy" + 0.017*"great" + 0.015*"thank" + 0.015*"amazing"')
(1, '0.040*"good" + 0.035*"great" + 0.033*"perfect" + 0.031*"small" + 0.024*"love" + 0.018*"light" + 0.013*"cake" + 0.013*"day" + 0.013*"magnet" + 0.013*"purchase"')
(2, '0.051*"good" + 0.018*"great" + 0.014*"time" + 0.013*"quality" + 0.013*"work" + 0.013*"small" + 0.013*"one" + 0.012*"excellent" + 0.011*"size" + 0.011*"thing"')
(3, '0.039*"product" + 0.038*"great" + 0.034*"good" + 0.033*"beautiful" + 0.026*"time" + 0.025*"happy" + 0.017*"topper" + 0.014*"life" + 0.014*"cake" + 0.014*"awesome"')
(4, '0.069*"great" + 0.023*"thing" + 0.018*"use" + 0.016*"light" + 0.016*"lathe" + 0.015*"time" + 0.015*"quality" + 0.014*"year" + 0.014*"old" + 0.014*"perfect"')
(5, '0.050*"good" + 0.036*"great" + 0.033*"helpful" + 0.017*"time" + 0.015*"nice" + 0.014*"excellent" + 0.013*"last" + 0.011*"satisfied" + 0.011*"amazing" + 0.010*"fit"')

In [57]:
lda_corpus

In [58]:
cluster1 = [j for i,j in zip(lda_corpus,documents) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,documents) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,documents) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,documents) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,documents) if i[4][1] > threshold]
cluster6 = [j for i,j in zip(lda_corpus,documents) if i[5][1] > threshold]
cluster7 = [j for i,j in zip(lda_corpus,documents) if i[6][1] > threshold]
cluster8 = [j for i,j in zip(lda_corpus,documents) if i[7][1] > threshold]
cluster9 = [j for i,j in zip(lda_corpus,documents) if i[8][1] > threshold]
cluster10 = [j for i,j in zip(lda_corpus,documents) if i[9][1] > threshold]
cluster11 = [j for i,j in zip(lda_corpus,documents) if i[10][1] > threshold]
cluster12 = [j for i,j in zip(lda_corpus,documents) if i[11][1] > threshold]

In [61]:
{print('"'+text+'"'+'\n') for text in cluster6 if len(text.split(' '))>2}


"Well Made !"

"Love this shirt, and it came in just in time for the finale!!  Fits as expected, and great quality!"

"I have used this hose one time an it seems to be a very good an strong hose. I am very satisfied."

"Hose was exactly as expected."

"Muy buen libro."

"I like that it's big enough to fit me properly. I am using it post surgical and helps so much more than what the hospital sends you home with."

"I can’t get the stud to fit or stay in place so I can pierce. I haven’t been able to use it at all."

"Has storage and hold water, light weight"

"These were exactly what I needed.  They work really well on anything I have placed them on.  My grandson applied them to his shotgun & rifle. A good value."

"They are just as advertised"

"He never met an adjective he couodn’t throw a handful of synonyms at alongside. Gets pretty fatiguing."

"The earings arent sharp and it didn't pierce"

"Was exactly what I wanted"

"It arrived as advertised"

"Looove this! Just what I needed!"


{None}